# Test guide star selection

**Question:** Down to which magnitude do we have to go to find at least one guide star for each Pasiphae survey field?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from astropy.coordinates import Angle, SkyCoord
import astropy.units as u
from astropy.io.votable import parse
from datetime import datetime
#import matplotlib.pyplot as plt
import numpy as np
import os
from pandas import DataFrame
import pickle
#import seaborn as sns

from fieldgrid import FieldGridIsoLat
from guidestars import GuideStarWalopN, GuideStarWalopS, load_gaia

## WALOP-S

### Create Southern field grid

In [3]:
grid = FieldGridIsoLat(verbose=1)
grid.load_params('grid_pasiphae_s.json')

Create fields..
    Done                                                    
Final number of fields: 61630
Grid parameters loaded from grid_pasiphae_s.json.


### Set up guide star selector

In [4]:
selector = GuideStarWalopS()

Define parameters:

In [7]:
scale_arcmin = 0.23 # arcmin / mm
scale = np.radians(scale_arcmin / 60) # rad / mm
circle_radius = np.sqrt((180/2+15)**2*2) # mm
circle_offset = 15 # mm
field_size = 180 # mm
cam_size = [6.4, 4.75] # mm
guide_area = [(90, -55), (125, -55), (125, 60), (60, 125), (-55, 125), (-55, 90), (90, 90)] ## mm
instr_rot = 0 # rad
limit_arcmin = 1 # arcmin
limit = limit_arcmin / scale_arcmin
home_pos = (125, 125)
scale_xy = 1 / scale # mm / rad
n_min = 2
n_max = 3

In [8]:
selector.set_params(
        circle_radius, circle_offset, field_size, cam_size, guide_area, home_pos, instr_rot=instr_rot, limit=limit, 
        scale=scale, scale_xy=scale_xy, n_min=n_min, n_max=n_max)

Save/load parameters to/from JSON file:

In [9]:
selector.save_params('guide_walop_s.json')

Guide parameters saved in: guide_walop_s.json


### Read initial set of brightest Gaia stars

In [14]:
dir_in = 'gaia/'
votable_files = 'gaia_g_mag12_b28.vot'

gaia_ra, gaia_dec, gaia_mag = load_gaia(votable_files, dir_in=dir_in, dec_uplim=1)
print('# stars:', gaia_mag.shape[0])

Read VOTable 1/2..
Read VOTable 2/2..
# stars: 300848


Separate bright stars:

In [15]:
gaia_mag_limit = 7
sel = gaia_mag < gaia_mag_limit
gaia_bright_ra = gaia_ra[sel]
gaia_bright_dec = gaia_dec[sel]
gaia_ra = gaia_ra[~sel]
gaia_dec = gaia_dec[~sel]
gaia_mag = gaia_mag[~sel]
print(f'{np.sum(sel)} bright stars separated.')

8766 bright stars separated.


Add bright stars that should be avoided:

In [16]:
selector.set_stars_to_avoid(np.radians(gaia_bright_ra), np.radians(gaia_bright_dec))

8766 coordinates of bright stars set that will be avoided in the guide camera.


Add stars:

In [17]:
selector.set_stars(np.radians(gaia_ra), np.radians(gaia_dec), gaia_mag)

292082 candidate stars added.
Magnitude range: 7.0 - 11.0


### Select guide stars

In [18]:
runtime_start = datetime.now()
guidestars = selector.select(fieldgrid=grid, verbose=2)
print('\nRuntime:', datetime.now() - runtime_start)

Iterate through field grid..
  done.                             


Results:
--------------------------------------------
Guide stars selected:         121961
Fields without guide stars:     6793 (11.0%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3
--------------------------------------------
No. of guide stars: No. of fields
 0: 6793
 1: 13825
 2: 14900
 3: 26112
--------------------------------------------

Runtime: 1:14:46.757847


Save guide stars and selector:

In [19]:
with open('pickles/guidestars_S.pickle', mode='wb') as f:
    pickle.dump(guidestars, f)
with open('pickles/selector_S.pickle', mode='wb') as f:
    pickle.dump(selector, f)

### Add fainter stars for selection

Go deeper:

In [20]:
votable_files = 'gaia_g_mag12-13_b28.vot'

gaia_ra, gaia_dec, gaia_mag = load_gaia(votable_files, dir_in=dir_in, dec_uplim=1)
print('# stars:', gaia_mag.shape[0])

Read VOTable 1/1..
# stars: 351856


In [21]:
runtime_start = datetime.now()
selector.set_stars(np.radians(gaia_ra), np.radians(gaia_dec), gaia_mag)
print('\nRuntime:', datetime.now() - runtime_start)

Overwriting previous stars..
351856 candidate stars added.
Magnitude range: 11.0 - 12.0

20618 field do not have enough guide stars. Search for more..
  done.                             


Results:
--------------------------------------------
Guide stars selected:         145167
Fields without guide stars:      718 (1.2%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3

Runtime: 0:20:28.243965


Go deeper:

In [22]:
votable_files = 'gaia_g_mag13-14_b28.vot'

gaia_ra, gaia_dec, gaia_mag = load_gaia(votable_files, dir_in=dir_in, dec_uplim=1)
print('# stars:', gaia_mag.shape[0])

Read VOTable 1/1..
# stars: 712469


In [23]:
runtime_start = datetime.now()
selector.set_stars(np.radians(gaia_ra), np.radians(gaia_dec), gaia_mag)
print('\nRuntime:', datetime.now() - runtime_start)

Overwriting previous stars..
712469 candidate stars added.
Magnitude range: 12.0 - 13.0

3487 field do not have enough guide stars. Search for more..
  done.                             


Results:
--------------------------------------------
Guide stars selected:         149242
Fields without guide stars:       11 (0.0%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3

Runtime: 0:06:16.478579


Final status:

In [26]:
selector.check_results(verbose=2)

Results:
--------------------------------------------
Guide stars selected:         149371
Fields without guide stars:        0 (0.0%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3
--------------------------------------------
No. of guide stars: No. of fields
 1:   1
 2: 35517
 3: 26112
--------------------------------------------


### Save guide stars

In [27]:
with open('pickles/guidestars_S.pickle', mode='wb') as f:
    pickle.dump(guidestars, f)
with open('pickles/selector_S.pickle', mode='wb') as f:
    pickle.dump(selector, f)

### Conclusion for WALOP-S

For WALOP-S we are able to identify at least one guide star for each field if we search down to **14 mag**. This is far from the limit of 16 mag.

## WALOP-N

### Create Nouthern field grid

In [3]:
grid = FieldGridIsoLat(verbose=1)
grid.load_params('grid_pasiphae_n.json')

Create fields..
    Done                                                    
Final number of fields: 83685
Grid parameters loaded from grid_pasiphae_n.json.


### Set up guide star selector

In [10]:
selector = GuideStarWalopN()

Define parameters:

In [11]:
guide_radius = 22.5 * 60 # arcsec
field_size = 30 * 60 # arcsec
cam_size = [72, 36] # arcsec
home_pos = np.radians(0) # radians
limit = 0 # arcsec
scale = np.radians(1 / 3600) # rad / arcsec
scale_rot = 180 / np.pi # degrees / radian
n_min = 2
n_max = 3

In [12]:
selector.set_params(
        guide_radius, field_size, cam_size, home_pos, limit=0,
        scale=scale, scale_rot=scale_rot, n_min=n_min, n_max=n_max)

Save/load parameters to/from JSON file:

In [13]:
selector.save_params('guide_walop_n.json')

Guide parameters saved in: guide_walop_n.json


### Read initial set of brightest Gaia stars

Helper function:

In [17]:
dir_in = 'gaia/'
votable_files = 'gaia_g_mag12_b28.vot'

gaia_ra, gaia_dec, gaia_mag = load_gaia(votable_files, dir_in=dir_in, dec_lolim=-1)
print('# stars:', gaia_mag.shape[0])

Read VOTable 1/2..
Read VOTable 2/2..
# stars: 270340


Separate bright stars:

In [18]:
gaia_mag_limit = 7
sel = gaia_mag < gaia_mag_limit
gaia_bright_ra = gaia_ra[sel]
gaia_bright_dec = gaia_dec[sel]
gaia_ra = gaia_ra[~sel]
gaia_dec = gaia_dec[~sel]
gaia_mag = gaia_mag[~sel]
print(f'{np.sum(sel)} bright stars separated.')

8149 bright stars separated.


Add bright stars that should be avoided:

In [19]:
selector.set_stars_to_avoid(np.radians(gaia_bright_ra), np.radians(gaia_bright_dec))

8149 coordinates of bright stars set that will be avoided in the guide camera.


Add stars:

In [20]:
selector.set_stars(np.radians(gaia_ra), np.radians(gaia_dec), gaia_mag)

262191 candidate stars added.
Magnitude range: 7.0 - 11.0


### Select guide stars

In [21]:
runtime_start = datetime.now()
guidestars = selector.select(fieldgrid=grid, verbose=2)
print('\nRuntime:', datetime.now() - runtime_start)

Iterate through field grid..
  done.                             


Results:
--------------------------------------------
Guide stars selected:          86388
Fields without guide stars:    30182 (36.1%)
--------------------------------------------
Median number of field stars:      1
Mean number of field stars:        1
Max number of field stars:         3
--------------------------------------------
No. of guide stars: No. of fields
 0: 30182
 1: 29333
 2: 15455
 3: 8715
--------------------------------------------

Runtime: 1:18:27.186767


Save guide stars and selector:

In [22]:
with open('pickles/guidestars_N.pickle', mode='wb') as f:
    pickle.dump(guidestars, f)
with open('pickles/selector_N.pickle', mode='wb') as f:
    pickle.dump(selector, f)

### Add fainter stars for selection

Go deeper:

In [23]:
votable_files = 'gaia_g_mag12-13_b28.vot'

gaia_ra, gaia_dec, gaia_mag = load_gaia(votable_files, dir_in=dir_in, dec_lolim=-1)
print('# stars:', gaia_mag.shape[0])

Read VOTable 1/1..
# stars: 304045


In [24]:
runtime_start = datetime.now()
selector.set_stars(np.radians(gaia_ra), np.radians(gaia_dec), gaia_mag)
print('\nRuntime:', datetime.now() - runtime_start)

Overwriting previous stars..
304045 candidate stars added.
Magnitude range: 11.0 - 12.0

59515 field do not have enough guide stars. Search for more..
  done.                             


Results:
--------------------------------------------
Guide stars selected:         135804
Fields without guide stars:    10248 (12.2%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3

Runtime: 0:51:10.350496


Go deeper:

In [25]:
votable_files = 'gaia_g_mag13-14_b28.vot'

gaia_ra, gaia_dec, gaia_mag = load_gaia(votable_files, dir_in=dir_in, dec_lolim=-1)
print('# stars:', gaia_mag.shape[0])

Read VOTable 1/1..
# stars: 614908


In [26]:
runtime_start = datetime.now()
selector.set_stars(np.radians(gaia_ra), np.radians(gaia_dec), gaia_mag)
print('\nRuntime:', datetime.now() - runtime_start)

Overwriting previous stars..
614908 candidate stars added.
Magnitude range: 12.0 - 13.0

30033 field do not have enough guide stars. Search for more..
  done.                             


Results:
--------------------------------------------
Guide stars selected:         167987
Fields without guide stars:     1447 (1.7%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3

Runtime: 0:45:12.908973


Go deeper:

In [27]:
votable_files = 'gaia_g_mag14-14.5_b28.vot'

gaia_ra, gaia_dec, gaia_mag = load_gaia(votable_files, dir_in=dir_in, dec_lolim=-1)
print('# stars:', gaia_mag.shape[0])

Read VOTable 1/1..
# stars: 1165886


In [28]:
runtime_start = datetime.now()
selector.set_stars(np.radians(gaia_ra), np.radians(gaia_dec), gaia_mag)
print('\nRuntime:', datetime.now() - runtime_start)

Overwriting previous stars..
1165886 candidate stars added.
Magnitude range: 13.0 - 14.0

6651 field do not have enough guide stars. Search for more..
  done.                             


Results:
--------------------------------------------
Guide stars selected:         175576
Fields without guide stars:       63 (0.1%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3

Runtime: 0:17:11.329346


Go deeper:

In [36]:
votable_files = 'gaia_g_mag14.5-15_b28.vot'

gaia_ra, gaia_dec, gaia_mag = load_gaia(votable_files, dir_in=dir_in, dec_lolim=-1)
print('# stars:', gaia_mag.shape[0])

Read VOTable 1/1..
# stars: 891686


In [37]:
runtime_start = datetime.now()
selector.set_stars(np.radians(gaia_ra), np.radians(gaia_dec), gaia_mag)
print('\nRuntime:', datetime.now() - runtime_start)

Overwriting previous stars..
891686 candidate stars added.
Magnitude range: 14.0 - 14.5

446 field do not have enough guide stars. Search for more..
  done.                             


Results:
--------------------------------------------
Guide stars selected:         176011
Fields without guide stars:       10 (0.0%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3

Runtime: 0:01:01.931836


Go deeper:

In [39]:
votable_files = 'gaia_g_mag15-15.4_b28.vot'

gaia_ra, gaia_dec, gaia_mag = load_gaia(votable_files, dir_in=dir_in, dec_lolim=-1)
print('# stars:', gaia_mag.shape[0])

Read VOTable 1/1..
# stars: 1160799


In [40]:
runtime_start = datetime.now()
selector.set_stars(np.radians(gaia_ra), np.radians(gaia_dec), gaia_mag)
print('\nRuntime:', datetime.now() - runtime_start)

Overwriting previous stars..
1160799 candidate stars added.
Magnitude range: 14.5 - 15.0

64 field do not have enough guide stars. Search for more..
  done.                             


Results:
--------------------------------------------
Guide stars selected:         176079
Fields without guide stars:        1 (0.0%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3

Runtime: 0:00:10.618434


Final status:

In [29]:
selector.check_results(verbose=2)

Results:
--------------------------------------------
Guide stars selected:         175576
Fields without guide stars:       63 (0.1%)
--------------------------------------------
Median number of field stars:      2
Mean number of field stars:        2
Max number of field stars:         3
--------------------------------------------
No. of guide stars: No. of fields
 0:  63
 1: 383
 2: 74524
 3: 8715
--------------------------------------------


### Save guide stars

In [43]:
with open('pickles/guidestars_N.pickle', mode='wb') as f:
    pickle.dump(guidestars, f)
with open('pickles/selector_N.pickle', mode='wb') as f:
    pickle.dump(selector, f)

### Conclusion for WALOP-N

For WALOP-N we are able to identify at least one guide star for each field if we search down to **15 mag**. This is well below the limit of 16 mag.